In [2]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from rapidfuzz import fuzz
import nltk
import string

In [14]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2') #pretrained model (isme parameters less hn)

In [11]:
df = pd.read_csv('/content/chatbot_database.csv')
df.head(10)

,question,answer
0,What is the price of this T-shirt?,$56
1,What are your opening hours?,We are open from 9 AM to 9 PM.
2,Where is your store located?,"We are located in Downtown Plaza, Main Street."
3,Do you have return policy?,"Yes, you can return items within 30 days with ..."
4,How can I track my order?,You can track your order using the tracking nu...
5,Is this available in other sizes?,"Yes, we have Small, Medium, Large, and XL avai..."
6,Do you offer cash on delivery?,"Yes, cash on delivery is available for all loc..."
7,How long does shipping take?,Standard delivery takes 3 to 5 business days.
8,Can I cancel my order?,"Yes, orders can be cancelled within 2 hours of..."
9,Do you have any discount or offers?,Yes! We currently offer 15% off on all items u...


In [12]:
def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    filtered = [w for w in tokens if w not in stopwords.words('english')]
    return " ".join(filtered)

In [15]:
df['preprocessed'] = df['question'].apply(preprocess)
df['embedding'] = df['preprocessed'].apply(lambda x: model.encode(x))

In [18]:
# Main query ko handle
def get_response(user_input, threshold=0.7):
    user_input_prep = preprocess(user_input)
    user_vec = model.encode(user_input_prep)

    # Smiliarity
    similarities = [cosine_similarity([user_vec], [vec])[0][0] for vec in df['embedding']]
    max_sim_index = int(np.argmax(similarities))
    max_sim_score = similarities[max_sim_index]

    if max_sim_score >= threshold:
        return df.iloc[max_sim_index]['answer']
    else:
        fuzz_scores = df['question'].apply(lambda q: fuzz.partial_ratio(user_input, q))
        best_fuzz_score = fuzz_scores.max()
        if best_fuzz_score > 80:
            return df.iloc[fuzz_scores.idxmax()]['answer']
        return "Sorry, Please rephrase bro?"


In [ ]:
# loop to run, ask ques, type exit
if __name__ == "__main__":
  print("Welcome to the Saeedan Tech Chatbot! Ask me anything. Type Exit or quit to finish")
  while True:
    user_query = input("You: ")
    if user_query.lower() in ["exit", "quit"]:
      print("Chatbot: babyeeee!")
      break
    reply = get_response(user_query)
    print("Chatbot:", reply)


Welcome to the Saeedan Tech Chatbot! Ask me anything. Type Exit or quit to finish
You: what's the price
Chatbot: $56
